# Lab 06: Azure Video Indexer

## Overview
Azure Video Indexer is an AI-powered service that extracts rich insights from videos. This notebook demonstrates how to upload videos, extract comprehensive insights including faces, text, keywords, topics, sentiments, transcripts, objects, and scenes.

## Topics Covered
- Uploading and indexing videos
- Extracting video insights (faces, text, keywords, topics, sentiments)
- Working with video transcripts
- Object and scene detection in videos
- Timeline analysis
- Using Video Indexer API programmatically
- Downloading and processing insights
- Creating video highlights and summaries
- Facial recognition (requires limited access approval)

## Prerequisites
- Azure Video Indexer account
- API key and account ID
- Video file: `responsible_ai.mp4` (included in this directory)

## Setup

In [ ]:
!pip install requests python-dotenv pandas matplotlib pillow -q

In [ ]:
import os
import json
import time
import requests
from pathlib import Path
from typing import Dict, List, Optional
from dotenv import load_dotenv
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, HTML, Video, Markdown

# Load configuration
load_dotenv('python/.env')

# Video Indexer configuration
VIDEO_INDEXER_ACCOUNT_ID = os.getenv("VIDEO_INDEXER_ACCOUNT_ID")
VIDEO_INDEXER_API_KEY = os.getenv("VIDEO_INDEXER_API_KEY")
VIDEO_INDEXER_LOCATION = os.getenv("VIDEO_INDEXER_LOCATION", "trial")  # or your region like "eastus"

# API endpoints
API_BASE_URL = f"https://api.videoindexer.ai"

# Video file
VIDEO_FILE = "responsible_ai.mp4"

if not VIDEO_INDEXER_ACCOUNT_ID or not VIDEO_INDEXER_API_KEY:
    print("⚠ Warning: Video Indexer credentials not found in .env file")
    print("Please add VIDEO_INDEXER_ACCOUNT_ID and VIDEO_INDEXER_API_KEY to your .env file")
else:
    print("✓ Video Indexer configuration loaded")

if os.path.exists(VIDEO_FILE):
    print(f"✓ Video file found: {VIDEO_FILE}")
else:
    print(f"⚠ Video file not found: {VIDEO_FILE}")

## 1. Authentication and Access Token

Get an access token to authenticate with the Video Indexer API.

In [ ]:
def get_access_token(account_id: str, location: str, api_key: str) -> str:
    """Get access token for Video Indexer API."""
    url = f"{API_BASE_URL}/auth/{location}/Accounts/{account_id}/AccessToken"
    
    params = {
        "allowEdit": "true"
    }
    
    headers = {
        "Ocp-Apim-Subscription-Key": api_key
    }
    
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()
    
    # Response is a quoted string, remove quotes
    token = response.json().strip('"')
    print("✓ Access token obtained")
    return token

# Get access token
if VIDEO_INDEXER_ACCOUNT_ID and VIDEO_INDEXER_API_KEY:
    try:
        access_token = get_access_token(
            VIDEO_INDEXER_ACCOUNT_ID,
            VIDEO_INDEXER_LOCATION,
            VIDEO_INDEXER_API_KEY
        )
        print(f"Token length: {len(access_token)} characters")
    except Exception as e:
        print(f"Error obtaining access token: {str(e)}")
        access_token = None
else:
    access_token = None
    print("Skipping authentication - credentials not configured")

## 2. Upload and Index Video

Upload a video file and start the indexing process.

In [ ]:
def upload_video(video_path: str, video_name: str, access_token: str, 
                 location: str, account_id: str, 
                 privacy: str = "Private") -> Dict:
    """Upload and index a video."""
    
    url = f"{API_BASE_URL}/{location}/Accounts/{account_id}/Videos"
    
    params = {
        "accessToken": access_token,
        "name": video_name,
        "privacy": privacy,
        "description": "Video uploaded for analysis"
    }
    
    # Read video file
    with open(video_path, "rb") as video_file:
        files = {"file": video_file}
        
        print(f"Uploading video: {video_name}...")
        response = requests.post(url, params=params, files=files)
        response.raise_for_status()
    
    result = response.json()
    video_id = result["id"]
    
    print(f"✓ Video uploaded successfully")
    print(f"  Video ID: {video_id}")
    print(f"  Indexing started...")
    
    return result

def check_indexing_status(video_id: str, access_token: str, 
                         location: str, account_id: str) -> Dict:
    """Check the indexing status of a video."""
    url = f"{API_BASE_URL}/{location}/Accounts/{account_id}/Videos/{video_id}/Index"
    
    params = {
        "accessToken": access_token
    }
    
    response = requests.get(url, params=params)
    response.raise_for_status()
    
    return response.json()

def wait_for_indexing(video_id: str, access_token: str, 
                      location: str, account_id: str, 
                      max_wait: int = 600) -> Dict:
    """Wait for video indexing to complete."""
    print("Waiting for indexing to complete...")
    start_time = time.time()
    
    while time.time() - start_time < max_wait:
        status = check_indexing_status(video_id, access_token, location, account_id)
        state = status.get("state")
        
        if state == "Processed":
            elapsed = time.time() - start_time
            print(f"✓ Indexing complete! (took {elapsed:.1f}s)")
            return status
        elif state == "Failed":
            print("✗ Indexing failed")
            return status
        
        print(f"  Status: {state}... (elapsed: {time.time() - start_time:.1f}s)")
        time.sleep(10)
    
    print(f"⚠ Timeout after {max_wait}s")
    return check_indexing_status(video_id, access_token, location, account_id)

# Example: Upload video
print("\n=== Video Upload and Indexing ===")
print("This process will:")
print("  1. Upload the video to Azure Video Indexer")
print("  2. Start automatic indexing")
print("  3. Extract insights (may take several minutes)")
print("\nNote: The responsible_ai.mp4 video is about responsible AI practices.")

In [ ]:
# Upload and index the video (uncomment to run)
if access_token and os.path.exists(VIDEO_FILE):
    try:
        upload_result = upload_video(
            VIDEO_FILE,
            "Responsible AI Video",
            access_token,
            VIDEO_INDEXER_LOCATION,
            VIDEO_INDEXER_ACCOUNT_ID
        )
        
        video_id = upload_result["id"]
        
        # Wait for indexing to complete
        indexed_video = wait_for_indexing(
            video_id,
            access_token,
            VIDEO_INDEXER_LOCATION,
            VIDEO_INDEXER_ACCOUNT_ID
        )
    except Exception as e:
        print(f"Error: {str(e)}")
        video_id = None
else:
    print("Skipping upload - credentials or video file not available")
    video_id = None

## 3. Extract Video Insights

Once indexing is complete, extract comprehensive insights from the video.

In [ ]:
def get_video_insights(video_id: str, access_token: str, 
                       location: str, account_id: str) -> Dict:
    """Get comprehensive insights for an indexed video."""
    url = f"{API_BASE_URL}/{location}/Accounts/{account_id}/Videos/{video_id}/Index"
    
    params = {
        "accessToken": access_token
    }
    
    response = requests.get(url, params=params)
    response.raise_for_status()
    
    return response.json()

def display_video_summary(insights: Dict):
    """Display a summary of video insights."""
    print("\n" + "="*80)
    print("VIDEO INSIGHTS SUMMARY")
    print("="*80)
    
    # Basic info
    print(f"\n📹 Video: {insights.get('name', 'N/A')}")
    print(f"Duration: {insights.get('durationInSeconds', 0)} seconds")
    print(f"State: {insights.get('state', 'N/A')}")
    
    # Get insights from videos array
    if 'videos' in insights and len(insights['videos']) > 0:
        video_insights = insights['videos'][0]['insights']
        
        # Transcript
        if 'transcript' in video_insights:
            print(f"\n📝 Transcript: {len(video_insights['transcript'])} segments")
        
        # Keywords
        if 'keywords' in video_insights:
            keywords = video_insights['keywords']
            print(f"\n🔑 Keywords ({len(keywords)}):")
            for kw in keywords[:10]:
                print(f"  - {kw['name']} (confidence: {kw.get('confidence', 0):.2f})")
        
        # Topics
        if 'topics' in video_insights:
            topics = video_insights['topics']
            print(f"\n📚 Topics ({len(topics)}):")
            for topic in topics[:5]:
                print(f"  - {topic['name']} (confidence: {topic.get('confidence', 0):.2f})")
        
        # Faces (Note: May require limited access approval)
        if 'faces' in video_insights:
            faces = video_insights['faces']
            print(f"\n👤 Faces detected: {len(faces)}")
            if len(faces) > 0:
                print("  Note: Facial recognition requires Limited Access approval")
        
        # Labels (visual content)
        if 'labels' in video_insights:
            labels = video_insights['labels']
            print(f"\n🏷️ Visual Labels ({len(labels)}):")
            for label in labels[:10]:
                print(f"  - {label['name']} (appears {len(label.get('appearances', []))} times)")
        
        # Sentiments
        if 'sentiments' in video_insights:
            sentiments = video_insights['sentiments']
            print(f"\n😊 Sentiments: {len(sentiments)} segments analyzed")
            
            # Count sentiment types
            sentiment_counts = {}
            for sentiment in sentiments:
                s_type = sentiment.get('sentimentType', 'Unknown')
                sentiment_counts[s_type] = sentiment_counts.get(s_type, 0) + 1
            
            for s_type, count in sentiment_counts.items():
                print(f"  - {s_type}: {count} segments")
        
        # Brands
        if 'brands' in video_insights:
            brands = video_insights['brands']
            if len(brands) > 0:
                print(f"\n🏢 Brands detected: {len(brands)}")
                for brand in brands[:5]:
                    print(f"  - {brand['name']}")
        
        # Named locations
        if 'namedLocations' in video_insights:
            locations = video_insights['namedLocations']
            if len(locations) > 0:
                print(f"\n📍 Locations mentioned: {len(locations)}")
                for loc in locations[:5]:
                    print(f"  - {loc['name']}")
    
    print("\n" + "="*80)

# Get and display insights
if video_id and access_token:
    try:
        insights = get_video_insights(
            video_id,
            access_token,
            VIDEO_INDEXER_LOCATION,
            VIDEO_INDEXER_ACCOUNT_ID
        )
        display_video_summary(insights)
    except Exception as e:
        print(f"Error getting insights: {str(e)}")
        insights = None
else:
    print("Skipping insights extraction - video not indexed")
    insights = None

## 4. Extract and Display Transcript

Get the full transcript with timestamps.

In [ ]:
def extract_transcript(insights: Dict) -> List[Dict]:
    """Extract transcript with timestamps."""
    if 'videos' not in insights or len(insights['videos']) == 0:
        return []
    
    video_insights = insights['videos'][0]['insights']
    
    if 'transcript' not in video_insights:
        return []
    
    transcript_segments = []
    
    for segment in video_insights['transcript']:
        transcript_segments.append({
            "id": segment.get('id'),
            "text": segment.get('text'),
            "confidence": segment.get('confidence'),
            "speaker_id": segment.get('speakerId'),
            "start": segment.get('start'),
            "end": segment.get('end')
        })
    
    return transcript_segments

def display_transcript(transcript: List[Dict], max_lines: int = 20):
    """Display transcript in readable format."""
    print("\n" + "="*80)
    print("VIDEO TRANSCRIPT")
    print("="*80 + "\n")
    
    for i, segment in enumerate(transcript[:max_lines]):
        timestamp = segment.get('start', 'N/A')
        text = segment.get('text', '')
        confidence = segment.get('confidence', 0)
        
        print(f"[{timestamp}] {text}")
        print(f"  (confidence: {confidence:.2f})\n")
    
    if len(transcript) > max_lines:
        print(f"... and {len(transcript) - max_lines} more segments")
    
    print("\n" + "="*80)

def get_full_transcript_text(transcript: List[Dict]) -> str:
    """Get full transcript as continuous text."""
    return " ".join([seg.get('text', '') for seg in transcript])

# Extract and display transcript
if insights:
    transcript = extract_transcript(insights)
    if transcript:
        display_transcript(transcript)
        
        # Get full text
        full_text = get_full_transcript_text(transcript)
        print(f"\nFull transcript length: {len(full_text)} characters")
        print(f"Total segments: {len(transcript)}")
    else:
        print("No transcript available")
else:
    print("No insights available")

## 5. Timeline Analysis

Analyze when different insights appear throughout the video.

In [ ]:
def analyze_timeline(insights: Dict) -> Dict:
    """Analyze insights timeline."""
    if 'videos' not in insights or len(insights['videos']) == 0:
        return {}
    
    video_insights = insights['videos'][0]['insights']
    duration = insights.get('durationInSeconds', 0)
    
    timeline = {
        "duration": duration,
        "keyword_timeline": [],
        "sentiment_timeline": [],
        "label_timeline": []
    }
    
    # Keywords over time
    if 'keywords' in video_insights:
        for keyword in video_insights['keywords']:
            for appearance in keyword.get('appearances', []):
                timeline['keyword_timeline'].append({
                    'keyword': keyword['name'],
                    'start': appearance.get('startSeconds', 0),
                    'end': appearance.get('endSeconds', 0)
                })
    
    # Sentiments over time
    if 'sentiments' in video_insights:
        for sentiment in video_insights['sentiments']:
            timeline['sentiment_timeline'].append({
                'sentiment': sentiment.get('sentimentType'),
                'start': sentiment.get('seenDuration', 0)
            })
    
    # Labels over time
    if 'labels' in video_insights:
        for label in video_insights['labels']:
            for appearance in label.get('appearances', []):
                timeline['label_timeline'].append({
                    'label': label['name'],
                    'start': appearance.get('startSeconds', 0),
                    'end': appearance.get('endSeconds', 0)
                })
    
    return timeline

def visualize_sentiment_timeline(timeline: Dict):
    """Visualize sentiment changes over time."""
    if not timeline.get('sentiment_timeline'):
        print("No sentiment data available")
        return
    
    sentiments = timeline['sentiment_timeline']
    
    # Create visualization
    fig, ax = plt.subplots(figsize=(14, 6))
    
    sentiment_colors = {
        'Positive': 'green',
        'Neutral': 'gray',
        'Negative': 'red'
    }
    
    for sentiment in sentiments:
        s_type = sentiment['sentiment']
        start = sentiment['start']
        color = sentiment_colors.get(s_type, 'blue')
        
        ax.scatter(start, 1, c=color, s=100, alpha=0.6, label=s_type)
    
    # Remove duplicate labels
    handles, labels = ax.get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    ax.legend(by_label.values(), by_label.keys())
    
    ax.set_xlabel('Time (seconds)')
    ax.set_title('Sentiment Analysis Timeline')
    ax.set_yticks([])
    ax.set_xlim(0, timeline['duration'])
    
    plt.tight_layout()
    plt.show()

# Analyze and visualize timeline
if insights:
    timeline = analyze_timeline(insights)
    
    print(f"\n=== Timeline Analysis ===")
    print(f"Video duration: {timeline['duration']} seconds")
    print(f"Keywords tracked: {len(timeline['keyword_timeline'])} occurrences")
    print(f"Sentiment changes: {len(timeline['sentiment_timeline'])}")
    print(f"Visual labels: {len(timeline['label_timeline'])} occurrences")
    
    # Visualize sentiments
    visualize_sentiment_timeline(timeline)
else:
    print("No insights available for timeline analysis")

## 6. Object and Scene Detection

Extract detected objects and scenes from the video.

In [ ]:
def extract_objects_and_scenes(insights: Dict) -> Dict:
    """Extract detected objects and scenes."""
    if 'videos' not in insights or len(insights['videos']) == 0:
        return {"labels": [], "scenes": []}
    
    video_insights = insights['videos'][0]['insights']
    
    # Labels represent visual objects
    labels = []
    if 'labels' in video_insights:
        for label in video_insights['labels']:
            labels.append({
                'name': label['name'],
                'id': label['id'],
                'appearances': len(label.get('appearances', [])),
                'total_duration': sum(
                    app.get('endSeconds', 0) - app.get('startSeconds', 0)
                    for app in label.get('appearances', [])
                )
            })
    
    # Scenes
    scenes = []
    if 'scenes' in video_insights:
        for scene in video_insights['scenes']:
            scenes.append({
                'id': scene['id'],
                'start': scene.get('instances', [{}])[0].get('start'),
                'end': scene.get('instances', [{}])[0].get('end')
            })
    
    return {"labels": labels, "scenes": scenes}

def display_top_objects(objects_data: Dict, top_n: int = 15):
    """Display most frequently appearing objects."""
    labels = objects_data['labels']
    
    if not labels:
        print("No object data available")
        return
    
    # Sort by total duration
    sorted_labels = sorted(labels, key=lambda x: x['total_duration'], reverse=True)
    
    print(f"\n=== Top {top_n} Visual Objects ===")
    print(f"{'Object':<30} {'Appearances':<15} {'Total Duration (s)'}")
    print("-" * 65)
    
    for label in sorted_labels[:top_n]:
        print(f"{label['name']:<30} {label['appearances']:<15} {label['total_duration']:.2f}")
    
    # Visualize top objects
    if len(sorted_labels) > 0:
        top_labels = sorted_labels[:top_n]
        names = [l['name'] for l in top_labels]
        durations = [l['total_duration'] for l in top_labels]
        
        fig, ax = plt.subplots(figsize=(12, 6))
        ax.barh(names, durations, color='steelblue')
        ax.set_xlabel('Total Duration (seconds)')
        ax.set_title('Top Visual Objects by Duration')
        ax.invert_yaxis()
        plt.tight_layout()
        plt.show()

# Extract and display objects
if insights:
    objects_data = extract_objects_and_scenes(insights)
    display_top_objects(objects_data)
    print(f"\nTotal scenes detected: {len(objects_data['scenes'])}")
else:
    print("No insights available")

## 7. Create Video Summary

Generate a comprehensive summary of the video content.

In [ ]:
def generate_video_summary(insights: Dict, transcript: List[Dict]) -> str:
    """Generate a comprehensive video summary."""
    if 'videos' not in insights or len(insights['videos']) == 0:
        return "No insights available"
    
    video_insights = insights['videos'][0]['insights']
    
    summary = []
    summary.append("# Video Analysis Summary")
    summary.append("\n## Basic Information")
    summary.append(f"- **Title**: {insights.get('name', 'N/A')}")
    summary.append(f"- **Duration**: {insights.get('durationInSeconds', 0)} seconds")
    summary.append(f"- **State**: {insights.get('state', 'N/A')}")
    
    # Key topics
    if 'topics' in video_insights:
        summary.append("\n## Main Topics")
        for topic in video_insights['topics'][:5]:
            summary.append(f"- {topic['name']}")
    
    # Keywords
    if 'keywords' in video_insights:
        summary.append("\n## Key Terms")
        keywords = [kw['name'] for kw in video_insights['keywords'][:10]]
        summary.append(", ".join(keywords))
    
    # Sentiment overview
    if 'sentiments' in video_insights:
        sentiment_counts = {}
        for sentiment in video_insights['sentiments']:
            s_type = sentiment.get('sentimentType', 'Unknown')
            sentiment_counts[s_type] = sentiment_counts.get(s_type, 0) + 1
        
        summary.append("\n## Sentiment Analysis")
        for s_type, count in sentiment_counts.items():
            summary.append(f"- {s_type}: {count} segments")
    
    # Transcript excerpt
    if transcript:
        summary.append("\n## Transcript Excerpt")
        excerpt = " ".join([seg['text'] for seg in transcript[:5]])
        summary.append(excerpt[:500] + "...")
    
    # Visual content
    if 'labels' in video_insights:
        summary.append("\n## Visual Content")
        top_labels = sorted(
            video_insights['labels'],
            key=lambda x: len(x.get('appearances', [])),
            reverse=True
        )[:10]
        labels_list = [label['name'] for label in top_labels]
        summary.append(", ".join(labels_list))
    
    return "\n".join(summary)

# Generate and display summary
if insights and transcript:
    summary = generate_video_summary(insights, transcript)
    display(Markdown(summary))
else:
    print("Cannot generate summary - insights or transcript not available")

## 8. Download Insights JSON

Save complete insights data for further processing.

In [ ]:
def save_insights_to_file(insights: Dict, filename: str = "video_insights.json"):
    """Save insights to a JSON file."""
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(insights, f, indent=2, ensure_ascii=False)
    
    file_size = os.path.getsize(filename)
    print(f"✓ Insights saved to {filename}")
    print(f"  File size: {file_size:,} bytes")
    
    return filename

# Save insights
if insights:
    output_file = save_insights_to_file(insights, "responsible_ai_insights.json")
else:
    print("No insights to save")

## 9. Search Within Video

Search for specific content within the video using the insights.

In [ ]:
def search_video_content(insights: Dict, transcript: List[Dict], 
                        query: str) -> List[Dict]:
    """Search for content within the video."""
    results = []
    query_lower = query.lower()
    
    # Search transcript
    for segment in transcript:
        if query_lower in segment['text'].lower():
            results.append({
                'type': 'transcript',
                'timestamp': segment['start'],
                'text': segment['text'],
                'confidence': segment.get('confidence', 0)
            })
    
    # Search keywords
    if 'videos' in insights and len(insights['videos']) > 0:
        video_insights = insights['videos'][0]['insights']
        
        if 'keywords' in video_insights:
            for keyword in video_insights['keywords']:
                if query_lower in keyword['name'].lower():
                    for appearance in keyword.get('appearances', []):
                        results.append({
                            'type': 'keyword',
                            'timestamp': appearance.get('startTime'),
                            'text': keyword['name'],
                            'confidence': keyword.get('confidence', 0)
                        })
    
    return results

def display_search_results(results: List[Dict], max_results: int = 10):
    """Display search results."""
    if not results:
        print("No results found")
        return
    
    print(f"\nFound {len(results)} results:")
    print("-" * 80)
    
    for i, result in enumerate(results[:max_results]):
        print(f"\n{i+1}. [{result['timestamp']}] ({result['type']})")
        print(f"   {result['text']}")
        print(f"   Confidence: {result.get('confidence', 0):.2f}")
    
    if len(results) > max_results:
        print(f"\n... and {len(results) - max_results} more results")

# Example search
if insights and transcript:
    search_query = "AI"  # Change this to search for different terms
    print(f"\n=== Searching for: '{search_query}' ===")
    search_results = search_video_content(insights, transcript, search_query)
    display_search_results(search_results)
else:
    print("No data available for search")

## Summary

This notebook demonstrated how to use Azure Video Indexer to:

1. **Upload and Index Videos** - Upload videos and extract comprehensive insights
2. **Extract Insights** - Get faces, keywords, topics, sentiments, and more
3. **Work with Transcripts** - Access timestamped transcriptions
4. **Timeline Analysis** - Track when insights appear throughout the video
5. **Object Detection** - Identify and track visual objects and scenes
6. **Generate Summaries** - Create comprehensive video summaries
7. **Save Insights** - Export insights for further processing
8. **Search Content** - Search within video content

### Important Notes

- **Facial Recognition**: Requires Limited Access approval from Microsoft
- **Processing Time**: Video indexing can take several minutes depending on video length
- **API Limits**: Be aware of rate limits and quotas for your subscription

### Next Steps

- Explore the Advanced Video Indexer notebook for more features
- Build custom models for brands and people
- Integrate with other Azure services
- Create automated video analysis pipelines

### Additional Resources

- [Azure Video Indexer Documentation](https://learn.microsoft.com/azure/azure-video-indexer/)
- [Video Indexer API Reference](https://api-portal.videoindexer.ai/)
- [Video Indexer Developer Portal](https://www.videoindexer.ai/)